In [1]:
import os
import requests

import geopandas as gpd
import geojsonio

import pandas as pd
from pandas.io.json import json_normalize

import folium
from folium import plugins

## Schema of data

## Getting data

In [2]:
# Option 1 is preferred: live data fed straight in
r = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson")
data = r.json()

In [3]:
# Option 2 - load a csv that was downloaded and saved before (used for the getting the folium map up intially)
# data = pd.read_csv('~/Code/Pybraries/folium/quakes/data/2.5_month.csv')

In [4]:
#data

## Parsing the data

In [5]:
quake_data = pd.json_normalize(data)
quake_data

,type,features,bbox,metadata.generated,metadata.url,metadata.title,metadata.status,metadata.api,metadata.count
0,FeatureCollection,"[{'type': 'Feature', 'properties': {'mag': 2.8...","[-179.7895, -23.3986, 0, 179.0896, 61.4535, 54...",1605838393000,https://earthquake.usgs.gov/earthquakes/feed/v...,"USGS Magnitude 2.5+ Earthquakes, Past Day",200,1.10.3,27


In [6]:
quake_features = pd.json_normalize(data=data['features'], meta=['mag','place','time','updated','tz','url','detail','felt','cdi','mmi','alert','status','tsunami','sig','net','code','ids','sources','types','nst','dmin','rms','gap','magType'])

In [15]:
quake_features.columns = quake_features.columns.map(lambda x: x.split(".")[-1])

quake_features

,type,id,mag,place,time,updated,tz,url,detail,felt,...,types,nst,dmin,rms,gap,magType,type,title,type,coordinates
0,Feature,nc73484551,2.85,"9km ENE of Hydesville, CA",1605835252870,1605836863037,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,12.0,...,",dyfi,nearby-cities,origin,phase-data,scitech-...",18.0,0.19350,0.1000,110.00,md,earthquake,"M 2.9 - 9km ENE of Hydesville, CA",Point,"[-124.0070038, 40.590332, 22.15]"
1,Feature,us6000cqlc,4.70,"13 km S of Piedecuesta, Colombia",1605833820415,1605835527492,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,1.0,...,",dyfi,origin,phase-data,",NaN,0.96700,0.6700,43.00,mb,earthquake,"M 4.7 - 13 km S of Piedecuesta, Colombia",Point,"[-73.0722, 6.8661, 146]"
2,Feature,us6000cql4,4.80,"23 km SSE of Yauri, Peru",1605831069987,1605831732040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",NaN,3.14000,1.2500,100.00,mb,earthquake,"M 4.8 - 23 km SSE of Yauri, Peru",Point,"[-71.3498, -14.9984, 151.22]"
3,Feature,hv72236807,2.59,"8 km ENE of Pāhala, Hawaii",1605822792250,1605825239040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",48.0,NaN,0.1100,96.00,md,earthquake,"M 2.6 - 8 km ENE of Pāhala, Hawaii",Point,"[-155.408, 19.2423333333333, 31.58]"
4,Feature,uw61691897,2.87,"1 km WSW of Onalaska, Washington",1605819708100,1605822940684,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,1.0,...,",dyfi,origin,phase-data,shakemap,",63.0,0.09609,0.1700,40.00,ml,earthquake,"M 2.9 - 1 km WSW of Onalaska, Washington",Point,"[-122.737666666667, 46.5708333333333, 22.43]"
5,Feature,ci39470639,3.06,"14km S of Arvin, CA",1605810143790,1605836445664,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,3.0,...,",dyfi,focal-mechanism,nearby-cities,origin,pha...",45.0,0.04587,0.1600,59.00,ml,earthquake,"M 3.1 - 14km S of Arvin, CA",Point,"[-118.8268333, 35.081, 2.71]"
6,Feature,us6000cqge,4.70,"99 km SW of Padang, Indonesia",1605809457336,1605821386040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",NaN,2.28400,0.6400,201.00,mb,earthquake,"M 4.7 - 99 km SW of Padang, Indonesia",Point,"[99.6307, -1.4843, 53.98]"
7,Feature,us6000cqg2,4.20,"21 km NNW of Fukagawa, Japan",1605806469675,1605820437040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",NaN,0.49700,0.6800,91.00,mb,earthquake,"M 4.2 - 21 km NNW of Fukagawa, Japan",Point,"[141.9748, 43.8986, 205.03]"
8,Feature,pr2020324010,2.99,"2 km SSW of La Parguera, Puerto Rico",1605802656310,1605803575903,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",15.0,0.01540,0.1000,229.00,md,earthquake,"M 3.0 - 2 km SSW of La Parguera, Puerto Rico",Point,"[-67.0548, 17.9583, 10]"
9,Feature,us6000cqeh,4.40,"73 km SW of Uyuni, Bolivia",1605800956775,1605823990040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,...,",origin,phase-data,",NaN,1.81100,0.8300,77.00,mb,earthquake,"M 4.4 - 73 km SW of Uyuni, Bolivia",Point,"[-67.3827, -20.8692, 195.62]"


I don't want to keep columns if they are not going to be needed but some might be. Time for some reading, thinking, and deciding where I might want this to go.

In [ ]:
df['url'].unique()

In [ ]:
m = folium.Map(
    location=[0, 0],
    tiles='cartodbpositron',
    zoom_start=1.5
)

In [ ]:
for i, row in quakes.iterrows():
    folium.CircleMarker((row.latitude, row.longitude), radius=row.mag*2.2, 
                        color='red', weight = 0, opacity =.4, fill=True, fill_color='orange', fill_opacity=.3, 
                        popup=[f"Time:{row.time},\n Mag: {row.mag},\n Depth: {row.depth} km"]).add_to(m)
    

In [ ]:
boundaries = ('data/GeoJSON/PB2002_boundaries.json')
line_style = {'color': '#FF3333', 'weight': 1, 'opacity':row.mag*1.4}


folium.GeoJson(
    boundaries,
    name='major fault lines',
    style_function=lambda x:line_style,
    smooth_factor=4.0,
      
).add_to(m)
m